<a href="https://colab.research.google.com/github/xjdeng/MiniGPT4-image-labeler/blob/main/server_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Base Requirements

In [1]:
#@title

!git clone https://github.com/oobabooga/text-generation-webui
%cd text-generation-webui
!git log -n 1 --format=%H
!pip install -r requirements.txt
!git clone https://github.com/Wojtab/minigpt-4-pipeline extensions/multimodal/pipelines/minigpt-4-pipeline
!pip install -r extensions/multimodal/pipelines/minigpt-4-pipeline/requirements.txt
!pip install -U gradio_client==0.2.7 flask-cloudflared

Cloning into 'text-generation-webui'...
remote: Enumerating objects: 9505, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 9505 (delta 159), reused 227 (delta 139), pack-reused 9220
Receiving objects: 100% (9505/9505), 3.24 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (6336/6336), done.
/content/text-generation-webui
63ece46213483b9b2692a9e4299cf3cd0ed7adb2
  Cloning https://github.com/huggingface/peft (to revision 03eb378eb914fbee709ff7c86ba5b1d033b89524) to /tmp/pip-req-build-d_ekj8hk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-d_ekj8hk
  Running command git rev-parse -q --verify 'sha^03eb378eb914fbee709ff7c86ba5b1d033b89524'
  Running command git fetch -q https://github.com/huggingface/peft 03eb378eb914fbee709ff7c86ba5b1d033b89524
  Running command git checkout -q 03eb378eb914fbee709ff7c86ba5b1d033b89524
  Resolved https://github.com/huggingface

Cloning into 'extensions/multimodal/pipelines/minigpt-4-pipeline'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 34 (delta 11), reused 31 (delta 11), pack-reused 0
Receiving objects: 100% (34/34), 26.11 KiB | 8.70 MiB/s, done.
Resolving deltas: 100% (11/11), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: gradio_client
    Found existing installation: gradio_client 0.2.5
    Uninstalling gradio_client-0.2.5:
      Successfully uninstalled gradio_client-0.2.5


# Install GPTQ Requirements

In [2]:
#@title

!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!git log -n 1 --format=%H
!python setup_cuda.py install
%cd ..
!cp -r GPTQ-for-LLaMa/* .

Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 818, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 818 (delta 107), reused 100 (delta 100), pack-reused 697
Receiving objects: 100% (818/818), 469.73 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (497/497), done.
/content/text-generation-webui/GPTQ-for-LLaMa
53a2aeac935b7eef0e1451eb7b465fea4f318e7a
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        **********************************************************************

# Install Model
You can go to https://huggingface.co and pick another Vicuna-based model if you wish and plug the path (without the https://huggingface.co part) below. Note that you may need an A100 Premium Colab GPU for models larger than 13B.

In [3]:
modelstr = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ" #@param {type:"string"}

!python download-model.py $modelstr
modelfolder = modelstr.replace("/","_")

100% 6.72k/6.72k [00:00<00:00, 39.6MiB/s]
100% 8.11G/8.11G [00:48<00:00, 167MiB/s]
100% 583/583 [00:00<00:00, 5.01MiB/s]
100% 132/132 [00:00<00:00, 1.15MiB/s]
100% 57.0/57.0 [00:00<00:00, 433kiB/s]
100% 435/435 [00:00<00:00, 3.69MiB/s]
100% 1.84M/1.84M [00:00<00:00, 8.42MiB/s]
100% 500k/500k [00:00<00:00, 541MiB/s]
100% 727/727 [00:00<00:00, 6.09MiB/s]


# Run the server
Take note of the URL after "**Starting non-streaming server at public url**".
You may see some error messages but if you don't see a URL after that text above, then please stop and restart the cell below. You'll need to copy the URL of the non-streaming server and paste it into the appropriate fields in the Client notebook. You may need to go to View -> Show/hide output in the next cell.

After you're done running the server, please go up to **Runtime -> Disconnect and delete runtime**.

Otherwise, the server will keep running idle until Google shuts it down and Google may restrict your Colab access in the future if it happens frequently.

In [ ]:
#@title
!python server.py --extensions multimodal --multimodal-pipeline minigpt4-13b --share --chat --loader gptq-for-llama --model_type llama --model $modelfolder --wbits 4 --groupsize 128 --auto-devices --public --public-api --sdp-attention

2023-07-20 20:15:53 WARNING:The gradio "share link" feature uses a proprietary executable to create a reverse tunnel. Use it with care.
2023-07-20 20:15:59.900810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-20 20:16:01 INFO:Loading TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ...
2023-07-20 20:16:01 INFO:Found the following quantized model: models/TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ/Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors
2023-07-20 20:16:54 WARNING:models/TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ/tokenizer_config.json is different from the original LlamaTokenizer file. It is either customized or outdated.
2023-07-20 20:16:54 WARNING:models/TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ/special_tokens_map.json is different from the original LlamaTokenizer file. It is either customized or outdated.
2023-07-20 20:16:54 INFO:Replaced attention with sdp_attention
2023-07-20 20:16:54 INFO:Loade